In [ ]:
import math, hashlib
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from IPython.display import display
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
import matplotlib.pyplot as plt
import random

# --- Prompt for IBM credentials and backend ---
api_key = input('Enter your IBM Quantum API key: ')
instance_name= input('Enter your Instance name: ')
backend_name = input('Enter backend name (e.g., ibm_brisbane): ')

# --- Try to connect to IBM Quantum backend ---
# --- Authenticate & get backend ---
try:
    service = QiskitRuntimeService(channel="ibm_cloud", token=api_key, instance=instance_name)
    backend = service.backend(backend_name)
    sampler = Sampler(backend)
    print(f"✅ Connected to {backend_name}")
except Exception as e:
    print(f"Error accessing IBM backend: {e}")
    print("Falling back to local AerSimulator.")
    backend = AerSimulator()

def chaos_fractal_counts(n_qubits, seed, shots, with_measure=True):
    qc = QuantumCircuit(n_qubits)
    qc.h(range(n_qubits))  # equal amplitudes

    # Chaotic + fractal phases
    for i in range(n_qubits):
        qc.rz(np.pi * np.sin(seed * (i + 1)**2), i)
    for i in range(n_qubits - 1):
        qc.rz(np.pi / (2 ** (i + 1)), i + 1)

    if with_measure:
        qc.cz(i, i + 1)
        qc.measure_all()
        qc_transpile = transpile(qc, backend)
        sampler = Sampler(backend)
        job = sampler.run([qc_transpile], shots=shots)
        job_id = job.job_id()  # Get job ID
        print(f"Job ID: {job_id}")
        result = job.result()
        counts = result[0].data.meas.get_counts()
        return counts, qc, None
    else:
        sv = Statevector.from_instruction(qc)
        return None, qc, sv


def entropy_and_fidelity(counts, n_qubits):
    total = sum(counts.values())
    k = 2 ** n_qubits
    probs = [counts.get(format(i, f'0{n_qubits}b'), 0)/total for i in range(k)]
    # Shannon entropy (bits)
    H = -sum(p * math.log2(p) for p in probs if p > 0)
    # Classical fidelity to uniform
    F = (sum(math.sqrt(p) for p in probs) / math.sqrt(k)) ** 2
    return H, F


def generate_hash(n_qubits, seed, bits):
    """Collect >=bits raw outcomes and return both the raw bitstream and SHA-256 digest."""
    shots = (bits + n_qubits - 1) // n_qubits
    counts, _, _ = chaos_fractal_counts(n_qubits, seed, shots)

    # Expand counts into a shot-like sequence
    outcomes = []
    for state, c in counts.items():
        outcomes.extend([state] * c)
    random.shuffle(outcomes)
    bitstream = ''.join(outcomes)
    bitstream = bitstream[:bits]  # truncate to requested length

    digest = hashlib.sha256(bitstream.encode()).hexdigest()
    return bitstream, digest


# --------- Example Run ----------
n_qubits = 4
seed = 0.5
shots = 4096
counts, qc, _ = chaos_fractal_counts(n_qubits, seed, shots,  with_measure=True)
H, F = entropy_and_fidelity(counts, n_qubits)

print("Counts (first few):", dict(list(counts.items())[:8]))
print(f"Entropy ≈ {H:.3f} bits (target ~{n_qubits})")
print(f"Fidelity to uniform ≈ {F:.3f} (target > 0.95)")

raw_bits, digest = generate_hash(n_qubits, seed, bits=128)
print("Raw bits (first 64):", raw_bits[:64])
print("SHA-256 digest:", digest)


# Create a single figure with two subplots
fig, axes = plt.subplots(1, 2, figsize=(15, 4))

# Draw circuit on first subplot
qc.draw(output="mpl", ax=axes[0])
axes[0].set_title("Quantum Circuit")

# Draw histogram on second subplot
plot_histogram(counts, ax=axes[1])
axes[1].set_title("Measurement Results")

# Show everything
plt.tight_layout()
plt.show()

_, qc, sv = chaos_fractal_counts(n_qubits, seed, shots, with_measure=False)
# Draw Bloch sphere separately
plot_bloch_multivector(sv, title="Bloch Spheres for all qubits")